In [ ]:
class TEMPcomp:
    '''
    class TEMPcomp
    
    Compare temperature data of FESOM-REcoM to PHC3 Atlas.
    
    -Specify depth_limit to define upper/lower boundaries of the layer.
    -Use layerwise = True to compare a set of depth layers.
    '''
    def __init__(self,resultpath,savepath,mesh,ncpath,first_year,last_year,
                 WOAvar='t_an',
                 mapproj='rob',
                 cmap = 'inferno',
                 savefig=False,
                 layerwise=False,
                 depth_array=(0,50,200,1000,2000,4000),
                 uplow=[0, 100],
                 cmap_extension='max',
                 verbose=True,
                 plotting=True,
                 output=False,
                 Taylor=True,
                 get_overview = False,
                 runname='fesom'):
        
        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.ncpath = ncpath
        self.fyear = first_year
        self.lyear = last_year
        self.WOAvar = WOAvar
        self.mapproj = mapproj
        self.cmap = cmap
        self.savefig = savefig
        self.layerwise = layerwise
        self.get_overview = get_overview
        self.depth_array = depth_array
        self.uplow = uplow
        self.cmap_extension = cmap_extension
        self.verbose = verbose
        self.plotting = plotting
        self.output = output
        self.Taylor = Taylor
        
        import matplotlib.pyplot as plt
        import numpy as np
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import pyfesom2 as pf
        from load_interp_WOA import WOAdata
        from Py_f2recom_toolbox import plt_Taylor_norm
        from Py_f2recom_toolbox import plt_Taylor_comp
        
        if self.mapproj == 'rob':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'pc':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'sp':
            box=[-180, 180, -90, -30]
        elif self.mapproj == 'np':
            box=[-180, 180, 60, 90]
        
        self.mapproj = pf.get_proj(self.mapproj)         
        years = np.arange(first_year,last_year+1,1)


        labelfesom = 'FESOM {0}-{1}'.format(self.fyear,self.lyear)
        unitfesom = 'T [$^{\circ}$C]'       

        # load data -------------------------------------------------------------------------------------
        fesom = pf.get_data(resultpath, "temp", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=self.verbose)

        # load WOA data  -------------------------------------------------------------------------------------
        WOA_input = WOAdata(self.runname,self.resultpath,self.mesh,self.ncpath,self.WOAvar, get_overview=self.get_overview)
        woa_int = WOA_input.woa_int    
        
        labelwoa = 'WOA'
        unitwoa = 'T [$^{\circ}$C]' 

        # apply sea mask to WOA as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        woa_int_ma = np.copy(woa_int)
        woa_int_ma[fesom == 0] = 0

        # plot WOA and FESOM ----------------------------------------------------------------------------------
        if(self.layerwise):
            if(self.depth_array == []):
                depth_array = (0,50,200,1000,2000,4000)
                
            for i in np.arange(len(depth_array)-1):
                if depth_array[i+1] < -np.max(WOA_input.layer_depths):
                    print('{0}m is too deep for climatology.\nPlease consider choosing max depth at {1}!\n***'.format(uplow[1],np.max(GLODAP_input.layer_depths)))
                
                uplow = [depth_array[i], depth_array[i+1]]

                fesom_mean = pf.layermean_data(fesom, mesh, uplow = uplow)
                woa_int_ma_mean = pf.layermean_data(woa_int_ma, mesh, uplow = uplow)
                
                if(self.verbose):
                    print('\nFESOM mean = {0:4.4f}, std = {1:4.4f}, min = {2:4.4f}, max = {3:4.4f}\nWOA mean = {4:4.4f}, std = {5:4.4f}, min = {6:4.4f}, max = {7:4.4f}'.format(
                        np.nanmean(fesom_mean),np.nanstd(fesom_mean),np.nanmin(fesom_mean),np.nanmax(fesom_mean),
                        np.nanmean(woa_int_ma_mean),np.nanstd(woa_int_ma_mean),np.nanmin(woa_int_ma_mean),np.nanmax(woa_int_ma_mean)))

                if plotting:
                    fig = plt.figure(figsize=(15,12), constrained_layout=False)
                    axes = fig.subplot_mosaic(
                                """
                                AB
                                CC
                                """,
                                gridspec_kw={'hspace': 0.1, 'wspace': 0.1, 'bottom': 0.03}, 
                                subplot_kw=dict(projection=self.mapproj))

                    m1 = axes['A']
                    levels = np.arange(-2,25,1)
                    f1 = pf.subplot(mesh, fig, m1, [fesom_mean],
                        levels = levels,
                        units=unitwoa, 
                        mapproj=self.mapproj, # robinson projection takes more time!
                        cmap = self.cmap,
                        cmap_extension=self.cmap_extension,
                        titles=labelfesom+'\n ({0}-{1} m m)'.format(uplow[0],uplow[1]),
                        box=box,
                        )

                    m2 = axes['B']
                    f2 = pf.subplot(mesh, fig, m2, [woa_int_ma_mean], 
                                levels = levels,
                                units=unitwoa, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = self.cmap,
                                cmap_extension=self.cmap_extension,
                                titles=labelwoa+'\n ({0}-{1} m m)'.format(uplow[0],uplow[1]),
                                box=box,
                               )

                    cbar1_ax = fig.add_axes([0.13, 0.53, 0.76, 0.02])
                    cbar1 = fig.colorbar(f1,
                                    cax = cbar1_ax, 
                                    orientation = 'horizontal',
                                    fraction=0.046, pad=0.04) 
                    cbar1.set_label(unitfesom, fontsize=18)
                    cbar1.ax.tick_params(labelsize=18)

                    m3 = axes['C']

                    levels_diff = np.arange(-5,5.25,0.25)
                    f3 = pf.subplot(mesh, fig, m3, [fesom_mean-woa_int_ma_mean], 
                                rowscol= (1,1),
                                levels = levels_diff,
                                units=unitwoa, 
                                mapproj=self.mapproj, # robinson projection takes more time!
                                cmap = 'RdBu_r',
                                cmap_extension='both',
                                titles='FESOM - WOA'+' ({0}-{1} m)'.format(uplow[0],uplow[1]),
                                box=box,
                               )
                    
                    m1.text(-0.12, 1.05, 'A', transform=m1.transAxes,
                        size=30, weight='bold')
                    m2.text(-0.12, 1.05, 'B', transform=m2.transAxes,
                        size=30, weight='bold')
                    m3.text(-0.12, 1.05, 'E', transform=m3.transAxes,
                        size=30, weight='bold')
                    
                    fig.subplots_adjust(bottom=0.02)
                    cbar2_ax = fig.add_axes([0.13, 0.001, 0.76, 0.02])
                    cbar2 = fig.colorbar(f3,
                                    cax = cbar2_ax, 
                                    orientation = 'horizontal',
                                    fraction=0.046, pad=0.04) 
                    cbar2.set_label(unitfesom, fontsize=18)
                    cbar2.ax.tick_params(labelsize=18)

                    # fig export  -------------------------------------------------------------------------------------
                    if(self.savefig==True):
                        plt.savefig(self.savepath+self.runname+'_'+'TEMP_WOA'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.png'.format(uplow[0], uplow[1]), 
                                dpi = 300, bbox_inches='tight')
                        plt.savefig(self.savepath+self.runname+'_'+'TEMP_WOA'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.pdf'.format(uplow[0], uplow[1]), 
                                bbox_inches='tight')
                    plt.show(block=False)
                    
                
                if Taylor:
                    # statistics  -------------------------------------------------------------------------------------
                    # preparation of datasets
                    if np.isnan(np.min(woa_int_ma)): print('WARNING: The interpolated WOA field contains NaNs at depth')
                    if np.isnan(np.min(fesom)): print('WARNING: The interpolated FESOM field contains NaNs at depth')

                    title = 'Normalized Taylor Diagram for Temperature'
                    plt_Taylor_comp(temp_int_ma,TEMPfesom,mask=True,title=title, depth_array=depth_array, mesh=mesh,verbose = self.verbose)
                    
                    
                    # fig export  -------------------------------------------------------------------------------------
                    if(self.savefig==True):
                        plt.savefig(self.savepath+self.runname+'_'+'TEMP_WOA_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.png'.format(uplow[0], uplow[1]), 
                                dpi = 300, bbox_inches='tight')
                        plt.savefig(self.savepath+self.runname+'_'+'TEMP_WOA_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.pdf'.format(uplow[0], uplow[1]), 
                                bbox_inches='tight')
                    plt.show(block=False)
                    
                if output:
                    print('Only return non-layerwise output')
        
        # mean over depth  -------------------------------------------------------------------------------------
        else:
            if uplow[1] < -np.max(WOA_input.layer_depths):
                    print('{0}m is too deep for climatology.\nPlease consider choosing max depth at {1}!\n***'.format(uplow[1],np.max(GLODAP_input.layer_depths)))

            fesom_mean = pf.layermean_data(fesom, mesh, uplow = uplow)
            woa_int_ma_mean = pf.layermean_data(woa_int_ma, mesh, uplow = uplow)

            if(self.verbose):
                print('\nFESOM mean = {0:4.4f}, std = {1:4.4f}, min = {2:4.4f}, max = {3:4.4f}\nWOA mean = {4:4.4f}, std = {5:4.4f}, min = {6:4.4f}, max = {7:4.4f}'.format(
                np.nanmean(fesom_mean),np.nanstd(fesom_mean),np.nanmin(fesom_mean),np.nanmax(fesom_mean),
                np.nanmean(woa_int_ma_mean),np.nanstd(woa_int_ma_mean),np.nanmin(woa_int_ma_mean),np.nanmax(woa_int_ma_mean)))
            
        
            if plotting:
                fig = plt.figure(figsize=(15,12), constrained_layout=False)
                axes = fig.subplot_mosaic(
                                """
                                AB
                                CC
                                """,
                                gridspec_kw={'hspace': 0.1, 'wspace': 0.1, 'bottom': 0.03}, 
                                subplot_kw=dict(projection=self.mapproj))
                    
                m1 = axes['A']
                levels = np.arange(-2,25,1)
                f1 = pf.subplot(mesh, fig, m1, [fesom_mean],
                            levels = levels,
                            units=unitwoa, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = self.cmap,
                            cmap_extension=self.cmap_extension,
                            titles=labelfesom+'\n ({0}-{1} m)'.format(uplow[0],uplow[1]),
                            box=box,
                           )
                    
                m2 = axes['B']
                f2 = pf.subplot(mesh, fig, m2, [woa_int_ma_mean], 
                            levels = levels,
                            units=unitwoa, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = self.cmap,
                            cmap_extension=self.cmap_extension,
                            titles=labelwoa+'\n ({0}-{1} m)'.format(uplow[0],uplow[1]),
                            box=box,
                           )
                    
                cbar1_ax = fig.add_axes([0.13, 0.53, 0.76, 0.02])
                cbar1 = fig.colorbar(f1,
                                cax = cbar1_ax, 
                                orientation = 'horizontal',
                                fraction=0.046, pad=0.04)
                cbar1.set_label(unitfesom, fontsize=18)
                cbar1.ax.tick_params(labelsize=18)
        
                m3 = axes['C']

                levels_diff = np.arange(-5,5.25,0.25)
                f3 = pf.subplot(mesh, fig, m3, [fesom_mean-woa_int_ma_mean], 
                            rowscol= (1,1),
                            levels = levels_diff,
                            units=unitwoa, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = 'RdBu_r',
                            cmap_extension='both',
                            titles='FESOM - WOA'+' ({0}-{1} m)'.format(uplow[0],uplow[1]),
                            box=box,
                           )
                
                fig.subplots_adjust(bottom=0.02)
                cbar2_ax = fig.add_axes([0.13, 0.001, 0.76, 0.02])
                cbar2 = fig.colorbar(f3,
                                cax = cbar2_ax, 
                                orientation = 'horizontal',
                                fraction=0.046, pad=0.04) 
                cbar2.set_label(unitfesom, fontsize=18)
                cbar2.ax.tick_params(labelsize=18)

                
                # fig export  -------------------------------------------------------------------------------------
                if(self.savefig==True):
                    plt.savefig(self.savepath+self.runname+'_'+'TEMP_WOA'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.png'.format(uplow[0], uplow[1]),
                            dpi = 300, bbox_inches='tight')
                    plt.savefig(self.savepath+self.runname+'_'+'TEMP_WOA'+'_'+str(years[0])+'to'+str(years[-1])+'_{0}-{1}m.png'.format(uplow[0], uplow[1]),
                            bbox_inches='tight')
                plt.show(block=False)
            
            if self.Taylor: 
                # statistics  -------------------------------------------------------------------------------------            
                # preparation of datasets
                if np.isnan(np.min(woa_int_ma_mean)): print('WARNING: The interpolated WOA field contains NaNs at depth')
                if np.isnan(np.min(fesom_mean)): print('WARNING: The interpolated FESOM field contains NaNs at depth')

                title = 'Taylor Diagram for T \n(mean over depth,  max = {0}-{1}m)'.format(uplow[0],uplow[1]),
                plt_Taylor_norm(woa_int_ma_mean,fesom_mean,mask=True,title=title, verbose = self.verbose)
                
                
                # fig export  -------------------------------------------------------------------------------------
                if(self.savefig==True):                
                    plt.savefig(self.savepath+self.runname+'_'+'TEMP_WOA_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                            dpi = 300, bbox_inches='tight')
                    plt.savefig(self.savepath+self.runname+'_'+'TEMP_WOA_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.pdf', 
                            bbox_inches='tight')
                plt.show(block=False)
        
            if output:
                    self.temp_fesom = fesom_mean
                    self.temp_phc   = woa_int_ma_mean